In [40]:
import pandas as pd
import sys
import os

In [41]:
import pandas as pd
import os

# 예시: 파일 경로 목록
file_info = [
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2015.11.16)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2016.05.16)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2016.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2017.05.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2017.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2018.05.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2018.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2019.05.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2019.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2020.05.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2020.11.16)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2021.05.17)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2021.11.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2022.05.16)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2022.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2023.05.15)_ko.xls"
]

dfs = []

for path in file_info:
    # 1) 파일명에서 연도/월 → 라벨 생성 (2015_11 같은 느낌)
    fname = os.path.basename(path)
    date_str = fname.split("재무제표(")[-1].split(")_ko")[0]   # '2015.11.16'
    year = date_str[:4]
    month = date_str[5:7]
    label = f"{year}_{month}"

    # 2) 엑셀 읽기 (위 4줄 머리글 제거)
    df = pd.read_excel(path, sheet_name="재무상태표", skiprows=4)

    # 첫 컬럼 이름을 '항목'으로 바꾸고 인덱스로
    first_col = df.columns[0]
    df = df.rename(columns={first_col: "항목"})
    df["항목"] = df["항목"].astype(str).str.strip()
    df = df.set_index("항목")

    # 완전히 빈 행 제거
    df = df.dropna(how="all")

    # 컬럼 이름 공백 제거
    df.columns = df.columns.astype(str).str.replace(r"\s+", "", regex=True)

    # ★ 첫 번째 데이터 컬럼만 DataFrame으로 추출 (중요: [[]] 사용)
    df1 = df.iloc[:, [0]].copy()       # ← [[]] 이라서 DataFrame
    # 인덱스 중복 제거 (같은 계정명이 여러 번 나오는 경우 첫 번째만 사용)
    df1 = df1[~df1.index.duplicated(keep="first")]

    # 컬럼 이름을 라벨로 교체
    df1.columns = [label]

    # 리스트에 저장
    dfs.append(df1)

# 3) 계정과목(인덱스) 기준으로 outer concat
result = pd.concat(dfs, axis=1, join="outer").sort_index()

# 여기서 인덱스 클리닝 추가
result.index = result.index.str.replace(r"\(.*?\)", "", regex=True)  # (주5,6,7,22) 이런거 제거
result.index = result.index.str.strip()  # 앞뒤 공백 제거
result = result[~result.index.duplicated(keep="first")]  # 중복 계정 제거 (선택)

# 이제 깔끔해진 데이터 확인
print(result.head(20))

# 엑셀 저장
result.to_excel(
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\hlmando_bs_merged.xlsx"
)


                      2015_11       2016_05       2016_11       2017_05  \
항목                                                                        
                   제 2 기 3분기말    제 3 기 1분기말    제 3 기 3분기말    제 4 기 1분기말   
기타금융자산                    NaN           NaN           NaN           NaN   
기타미지급채무                   NaN           NaN           NaN           NaN   
기타부채              23813418093   21556093175   30790610902   35358202624   
기타수취채권                    NaN           NaN           NaN           NaN   
기타자본             194096618040  194096618040  194096618040  194096618040   
기타자산              62009765727   39994267772   25028763805   32145108623   
기타지급채무            32671948211   30846135468   32432721868   31216682687   
기타포괄손익-공정가치금융자산           NaN           NaN           NaN           NaN   
기타포괄손익누계액                 NaN           NaN           NaN           NaN   
단기리스부채                    NaN           NaN           NaN           NaN   
단기차입금                    

In [42]:
import pandas as pd
import sys

# 엑셀 파일들(정정후) => 4개
file_paths = [
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[(주)만도]3분기보고서_재무제표(2023.11.14)_ko.xlsx",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[에이치엘만도㈜]1분기보고서_재무제표(2024.05.16)_ko.xlsx",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[에이치엘만도㈜]3분기보고서_재무제표(2024.11.14)_ko.xlsx",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[에이치엘만도㈜]1분기보고서_재무제표(2025.05.15)_ko.xlsx"
]
# print(len(file_paths))

dfs = []
for i, path in enumerate(file_paths):
    df = pd.read_excel(path, sheet_name="D210005", skiprows=4)
    
    # 첫 번째 컬럼명 변경
    df.rename(columns={df.columns[0]: "항목"}, inplace=True)
    
    # 항목 컬럼 공백 제거
    df["항목"] = (
        df["항목"]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", "", regex=True)
    )

    # 파일명에서 기준일(날짜) 추출 → 열 이름 구분용
    date_tag = path.split("재무제표(")[-1].split(")_ko.xlsx")[0]

    if i == 0:
        # ✅ 첫 번째 파일: 항목 살리고, 나머지 컬럼에만 날짜 태그 붙이기
        value_cols = df.columns[1:]
        new_cols = ["항목"] + [f"{c}_{date_tag}" for c in value_cols]
        df.columns = new_cols
        dfs.append(df)
    else:
        # ✅ 나머지 파일: 항목(첫 번째 컬럼)은 버리고, 2번째 컬럼부터만 사용
        value_cols = df.columns[1:]
        df_values = df[value_cols].copy()
        df_values.columns = [f"{c}_{date_tag}" for c in value_cols]
        dfs.append(df_values)

# 옆으로 병합 (행 순서 동일하다고 가정)
df_merged = pd.concat(dfs, axis=1)

# 항목 기준 정렬
df_merged = df_merged.sort_values("항목").reset_index(drop=True)

print(df_merged.head(10))

# 엑셀로 저장
save_path = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\mando_merged1.xlsx"
df_merged.to_excel(save_path, index=False)

print(f"\n✅ 병합 완료! 엑셀로 저장됨 → {save_path}")


          항목  2023-09-30_2023.11.14  2022-12-31_2023.11.14  \
0  기타비유동금융자산           2.250310e+09           2.916990e+09   
1    기타비유동부채           2.062533e+09           1.557427e+09   
2    기타비유동자산           1.668348e+08           2.034879e+08   
3     기타수취채권           1.090773e+11           1.043078e+11   
4     기타수취채권           7.517636e+10           7.083373e+10   
5   기타유동금융자산           1.375000e+09           1.375000e+09   
6     기타유동부채           3.095031e+10           2.747699e+10   
7     기타유동자산           6.552726e+10           3.131284e+10   
8   기타자본구성요소           1.993695e+11           1.972320e+11   
9     기타지급채무           2.722836e+10           2.687726e+10   

   2024-03-31_2024.05.16  2023-12-31_2024.05.16  2024-09-30_2024.11.14  \
0           5.636337e+10           5.524531e+10           6.029112e+10   
1           6.004043e+09           5.714491e+09           1.836377e+12   
2           4.422361e+10           4.495775e+10           3.992312e+10   
3           1.312675e

c:\Users\qjadl\anaconda3\envs\DATA_311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\qjadl\anaconda3\envs\DATA_311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\qjadl\anaconda3\envs\DATA_311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\qjadl\anaconda3\envs\DATA_311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
import pandas as pd

file1 = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\hlmando_bs_merged.xlsx"
file2 = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\mando_merged1.xlsx"

df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)



# 4) '항목' 기준으로 병합
# how='outer'  : 두 파일의 항목 전체 포함 (겹치는 건 한 줄로, 나머진 NaN)
df_merged = pd.merge(df1, df2, on='항목', how='outer')

df_merged = df_merged.sort_values('항목').reset_index(drop=True)



# 6) 엑셀로 저장
save_path = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\mando_merged.xlsx"
df_merged.to_excel(save_path, index=False)

print(f"\n✅ 병합 완료! '항목' 이름이 같은 것만 묶어서 병합됨 → {save_path}")




✅ 병합 완료! '항목' 이름이 같은 것만 묶어서 병합됨 → C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\mando_merged.xlsx
